In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [130]:
filename = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"

In [131]:
df_master = pd.read_csv (filename)

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [132]:
df_master.head(5)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [133]:
df_master['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [134]:
#Replace blank value with NaN

df_master.replace (" ", np.nan, inplace=True)

In [135]:
# To check for missing data in table

missing_data = df_master.isnull()
missing_data.head(10)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,True,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False


In [136]:
#To count missing values in each column
#Use for loop method 

for column in missing_data.columns.values.tolist():
  print (column)
  print (missing_data[column].value_counts())
  print ("")


SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64

X
False    189339
True       5334
Name: X, dtype: int64

Y
False    189339
True       5334
Name: Y, dtype: int64

OBJECTID
False    194673
Name: OBJECTID, dtype: int64

INCKEY
False    194673
Name: INCKEY, dtype: int64

COLDETKEY
False    194673
Name: COLDETKEY, dtype: int64

REPORTNO
False    194673
Name: REPORTNO, dtype: int64

STATUS
False    194673
Name: STATUS, dtype: int64

ADDRTYPE
False    192747
True       1926
Name: ADDRTYPE, dtype: int64

INTKEY
True     129603
False     65070
Name: INTKEY, dtype: int64

LOCATION
False    191996
True       2677
Name: LOCATION, dtype: int64

EXCEPTRSNCODE
True     189035
False      5638
Name: EXCEPTRSNCODE, dtype: int64

EXCEPTRSNDESC
True     189035
False      5638
Name: EXCEPTRSNDESC, dtype: int64

SEVERITYCODE.1
False    194673
Name: SEVERITYCODE.1, dtype: int64

SEVERITYDESC
False    194673
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    189769
True       4904
Name: C

In [137]:
# Summary of missing values and counts (if True exists)

# 1. X True       5334
# 2. Y True       5334
# 3. ADDRTYPE True       1926
# 4. INTKEY True     129603
# 5. LOCATION True       2677
# 6. EXCEPTRSNCODE True     109862
# 7. EXCEPTRSNDESC rue     189035
# 8. COLLISIONTYPE True       4904
# 9. JUNCTIONTYPE True       6329
# 10. INATTENTIONIND True     164868
# 11. UNDERINFL True       4884
# 12. WEATHER True       5081
# 13. LIGHTCOND True       5170
# 14. PEDROWNOTGRNT True     190006
# 15. SDOTCOLNUM True      79737
# 16. SPEEDING True     185340
# 17. ST_COLCODE True       4904
# 18. ST_COLDESC True       4904

 



In [138]:
#Drop "INTKEY", "EXCEPTRSNCODE", "EXCEPTRSNDESC" and "INATTENTIONIND" as there are many unfilled cells 

df_master.drop(["INTKEY", "EXCEPTRSNCODE", "EXCEPTRSNDESC", "INATTENTIONIND"], axis=1, inplace=True)

In [139]:
df_master.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'LOCATION', 'SEVERITYCODE.1', 'SEVERITYDESC',
       'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT',
       'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [140]:
df_master.shape

(194673, 34)

In [141]:
# replace with frequency for "ADDRTYPE"

df_master['ADDRTYPE'].value_counts()

Block           126926
Intersection     65070
Alley              751
Name: ADDRTYPE, dtype: int64

In [142]:
#Replace NaN with 'Block' for "ADDRTYPE"

df_master["ADDRTYPE"].replace(np.nan, "Block", inplace=True)


In [143]:
df_master["ADDRTYPE"]

0         Intersection
1                Block
2                Block
3                Block
4         Intersection
              ...     
194668           Block
194669           Block
194670    Intersection
194671    Intersection
194672           Block
Name: ADDRTYPE, Length: 194673, dtype: object

In [144]:
#Caculate frequency of values for "COLLISIONTYPE" 

df_master["COLLISIONTYPE"].value_counts()

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         23703
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

In [145]:
#Replace Nan with "Parked car" for "COLLISIONTYPE"

df_master["COLLISIONTYPE" ].replace(np.nan, "Parked Car", inplace=True)
df_master["COLLISIONTYPE"]

0             Angles
1          Sideswipe
2         Parked Car
3              Other
4             Angles
             ...    
194668       Head On
194669    Rear Ended
194670     Left Turn
194671        Cycles
194672    Rear Ended
Name: COLLISIONTYPE, Length: 194673, dtype: object

In [146]:
#Caculate frequency of values for "JUNCTIONTYPE"

df_master["JUNCTIONTYPE"].value_counts()

Mid-Block (not related to intersection)              89800
At Intersection (intersection related)               62810
Mid-Block (but intersection related)                 22790
Driveway Junction                                    10671
At Intersection (but not related to intersection)     2098
Ramp Junction                                          166
Unknown                                                  9
Name: JUNCTIONTYPE, dtype: int64

In [147]:
#Replace Nan with "Mid-Block (not related to intersection)"

df_master["JUNCTIONTYPE"].replace(np.nan, "Mid-Block (not related to intersection)", inplace=True)
df_master["JUNCTIONTYPE"].isnull().value_counts()

False    194673
Name: JUNCTIONTYPE, dtype: int64

In [148]:
#Remove "X" and "Y" rows since coordinates are very specific 

df_master.dropna(subset=["X"], axis=0, inplace=True)

In [149]:
#Confirm number of missing data again

missing_data = df_master.isnull()
for column in missing_data.columns.values.tolist():
  print (column)
  print (missing_data[column].value_counts())
  print ("")


SEVERITYCODE
False    189339
Name: SEVERITYCODE, dtype: int64

X
False    189339
Name: X, dtype: int64

Y
False    189339
Name: Y, dtype: int64

OBJECTID
False    189339
Name: OBJECTID, dtype: int64

INCKEY
False    189339
Name: INCKEY, dtype: int64

COLDETKEY
False    189339
Name: COLDETKEY, dtype: int64

REPORTNO
False    189339
Name: REPORTNO, dtype: int64

STATUS
False    189339
Name: STATUS, dtype: int64

ADDRTYPE
False    189339
Name: ADDRTYPE, dtype: int64

LOCATION
False    189339
Name: LOCATION, dtype: int64

SEVERITYCODE.1
False    189339
Name: SEVERITYCODE.1, dtype: int64

SEVERITYDESC
False    189339
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    189339
Name: COLLISIONTYPE, dtype: int64

PERSONCOUNT
False    189339
Name: PERSONCOUNT, dtype: int64

PEDCOUNT
False    189339
Name: PEDCOUNT, dtype: int64

PEDCYLCOUNT
False    189339
Name: PEDCYLCOUNT, dtype: int64

VEHCOUNT
False    189339
Name: VEHCOUNT, dtype: int64

INCDATE
False    189339
Name: INCDATE, dtype: int

In [150]:
df_master.columns.value_counts()

PERSONCOUNT       1
VEHCOUNT          1
INCDTTM           1
ST_COLDESC        1
UNDERINFL         1
PEDCYLCOUNT       1
JUNCTIONTYPE      1
SEVERITYCODE.1    1
Y                 1
COLDETKEY         1
SPEEDING          1
SDOTCOLNUM        1
OBJECTID          1
LIGHTCOND         1
CROSSWALKKEY      1
PEDCOUNT          1
HITPARKEDCAR      1
SEGLANEKEY        1
INCDATE           1
ROADCOND          1
SEVERITYCODE      1
REPORTNO          1
PEDROWNOTGRNT     1
SEVERITYDESC      1
X                 1
WEATHER           1
ADDRTYPE          1
SDOT_COLCODE      1
COLLISIONTYPE     1
SDOT_COLDESC      1
LOCATION          1
ST_COLCODE        1
STATUS            1
INCKEY            1
dtype: int64

In [151]:
# Drop also "PEDROWNOTGRNT","SPEEDING" as there were more than 180,000 counts. Drop "SDOTCOLNUM" as it does not add any value to the analysis

df_master.drop(["PEDROWNOTGRNT", "SPEEDING", "SDOTCOLNUM"], axis=1, inplace=True)

In [152]:
#Calculate frequency of values for "WEATHER"

df_master["WEATHER"].value_counts()

Clear                       108959
Raining                      32015
Overcast                     27136
Unknown                      13893
Snowing                        894
Other                          773
Fog/Smog/Smoke                 553
Sleet/Hail/Freezing Rain       112
Blowing Sand/Dirt               50
Severe Crosswind                24
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [153]:
#Replace Nan with "Clear"

df_master["WEATHER"].replace(np.nan, "Clear", inplace=True)
df_master["WEATHER"].isnull().value_counts()

False    189339
Name: WEATHER, dtype: int64

In [154]:
#Calculate frequency of values for "WEATHER"

df_master["LIGHTCOND"].value_counts()

Daylight                    113582
Dark - Street Lights On      47314
Unknown                      12432
Dusk                          5775
Dawn                          2422
Dark - No Street Lights       1451
Dark - Street Lights Off      1152
Other                          188
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [155]:
#Replace Nan with "Daylight"

df_master["LIGHTCOND"].replace(np.nan, "Daylight", inplace=True)
df_master["LIGHTCOND"].isnull().value_counts()

False    189339
Name: LIGHTCOND, dtype: int64

In [156]:
#Remove "ST_COLCODE" and "ST_COLDESC" rows since coordinates are very specific 

df_master.dropna(subset=["ST_COLCODE"], axis=0, inplace=True)

In [157]:
#For "UNDERINFL", mix of "Y" and "N" and 1 and 0 found. Change "Y" to 1 and "No" to 0 

df_master["UNDERINFL"].replace("Y", "1", inplace=True)

In [158]:
df_master["UNDERINFL"].replace("N", "0", inplace=True)

In [159]:
# Remove rows with empty "UNDERINFL" values as they are specific to the analysis

df_master.dropna(subset=["UNDERINFL"], axis=0, inplace=True)

In [160]:
df_master["UNDERINFL"].value_counts()

0    175701
1      8880
Name: UNDERINFL, dtype: int64

In [161]:
#Final check on missing data

missing_data = df_master.isnull()
for column in missing_data.columns.values.tolist():
  print (column)
  print (missing_data[column].value_counts())
  print ("")

SEVERITYCODE
False    184581
Name: SEVERITYCODE, dtype: int64

X
False    184581
Name: X, dtype: int64

Y
False    184581
Name: Y, dtype: int64

OBJECTID
False    184581
Name: OBJECTID, dtype: int64

INCKEY
False    184581
Name: INCKEY, dtype: int64

COLDETKEY
False    184581
Name: COLDETKEY, dtype: int64

REPORTNO
False    184581
Name: REPORTNO, dtype: int64

STATUS
False    184581
Name: STATUS, dtype: int64

ADDRTYPE
False    184581
Name: ADDRTYPE, dtype: int64

LOCATION
False    184581
Name: LOCATION, dtype: int64

SEVERITYCODE.1
False    184581
Name: SEVERITYCODE.1, dtype: int64

SEVERITYDESC
False    184581
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    184581
Name: COLLISIONTYPE, dtype: int64

PERSONCOUNT
False    184581
Name: PERSONCOUNT, dtype: int64

PEDCOUNT
False    184581
Name: PEDCOUNT, dtype: int64

PEDCYLCOUNT
False    184581
Name: PEDCYLCOUNT, dtype: int64

VEHCOUNT
False    184581
Name: VEHCOUNT, dtype: int64

INCDATE
False    184581
Name: INCDATE, dtype: int

In [167]:
#Given that our target variable only has 2 values , 1 or 2, changing to 0 to 1 makes it look more like a binary. therefore now 0 = prop damage and 1 = injury 

df_master["SEVERITYCODE"].replace(1, "0", inplace=True)

In [168]:
df_master["SEVERITYCODE"].replace(2, "1", inplace=True)

In [169]:
df_master["SEVERITYCODE"].value_counts()

0    128476
1     56105
Name: SEVERITYCODE, dtype: int64

In [170]:
df_master.shape

(184581, 31)

<bound method NDFrame.describe of        SEVERITYCODE           X          Y  OBJECTID  INCKEY  COLDETKEY  \
0                 1 -122.323148  47.703140         1    1307       1307   
1                 0 -122.347294  47.647172         2   52200      52200   
2                 0 -122.334540  47.607871         3   26700      26700   
3                 0 -122.334803  47.604803         4    1144       1144   
4                 1 -122.306426  47.545739         5   17700      17700   
...             ...         ...        ...       ...     ...        ...   
194668            1 -122.290826  47.565408    219543  309534     310814   
194669            0 -122.344526  47.690924    219544  309085     310365   
194670            1 -122.306689  47.683047    219545  311280     312640   
194671            1 -122.355317  47.678734    219546  309514     310794   
194672            0 -122.289360  47.611017    219547  308220     309500   

       REPORTNO   STATUS      ADDRTYPE  \
0       3502005  Matche